### Imports

In [ ]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# import sparseconvnet.legacy as scn

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env TMP="/tmp"
%env JOBLIB_TEMP_FOLDER="/tmp"

In [ ]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [ ]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TRAIN_LABEL = '../data/train.csv'
FILETYPE = '.png'
img_size = 640

use_gpu = torch.cuda.is_available()

### Dataset class

In [ ]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, img_size, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype
        self.img_size = img_size
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
        img = img.point(lambda x: x if x<200 else 0)
#         print('PIL size:', img.size)
        img = np.asarray(img)
#         print('Np size: ',img.shape)
        if img.shape[1] > img.shape[0]:
#             print(img.shape)
            img = np.swapaxes(img,0,1)
        img = cv2.resize(img, (740,950))
        img = img[210:,:]
#         print(img.shape)
        rot_ang = range(0,360,30)
        rot_ang = rot_ang[np.random.randint(0,len(rot_ang))]
#         print(rot_ang)
        scale = np.random.uniform(0.8,1.2)
#         print(scale)
        rot_mat = cv2.getRotationMatrix2D((img.shape[0]/2,img.shape[1]/2), rot_ang, scale);
        img = cv2.warpAffine(img, rot_mat, (img.shape[0],img.shape[1]))
        img_he = self.clahe.apply(cv2.convertScaleAbs(img[:,:,0]))
        img_mask = cv2.adaptiveThreshold(img_he, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,
                                         np.random.randint(5,9) // 2 * 2 + 1, np.random.randint(5,9))
        img_ch = img_he * img_mask
        
        img = img_ch
#         for i in xrange(img.shape[2]):
#             img[:,:,i] = img_ch

        img = Image.fromarray(np.uint8(img))
        
#         image = cv2.imread(img_name)[:,:,::-1]
        boneage = self.labels.iloc[idx]['boneage'].astype(np.float32)
        gender = self.labels.iloc[idx]['male'].astype(np.uint8)
        
        label = np.hstack((boneage, gender))

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [ ]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.Scale(300),
#         transforms.CenterCrop(384),
        transforms.RandomSizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
#         transforms.Scale(300),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

### Data loaders

In [ ]:
labels = pd.read_csv(TRAIN_LABEL)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [ ]:
skf = StratifiedKFold(n_splits=4)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage', 'male')],
        'val': labels.loc[test, ('id', 'boneage', 'male')]
    }
    splits.append(fold_label)

In [ ]:
fold_label = splits[0]

In [ ]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, img_size, data_transforms[x])
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=24, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
#     for i in xrange(sample['image'].size()[0]):
#         img = sample['image'][i,0,:,:].numpy()
#         plt.figure()
#         plt.imshow(img, cmap=plt.cm.gray)
    if i_batch == 2:
        break

### Training

In [ ]:
class Net(nn.Module):
    def __init__(self, input_shape=(1, img_size, img_size)):
        super(Net, self).__init__()
        
#         self.bn = nn.BatchNorm2d(input_shape[0])
#         net = models.vgg16(pretrained=False)
        self.net = nn.DataParallel(nn.Sequential(
            nn.Conv2d(input_shape[0], 192, 5, stride=3),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.Conv2d(192, 160, 1, stride=1),
            nn.BatchNorm2d(160),
            nn.ReLU(),
            nn.Conv2d(160, 96, 1, stride=1),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            
            nn.MaxPool2d(3, stride=2, ceil_mode=True),
            nn.Dropout(),
            
            nn.Conv2d(96, 192, 5, stride=2),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            
            nn.AvgPool2d(3, stride=2, ceil_mode=True),
            nn.Dropout(),
            
            nn.Conv2d(192, 192, 3), stride=2),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.Conv2d(192, 1, 1, stride=1),
            nn.BatchNorm2d(1),
            nn.ReLU(),
            
            nn.AvgPool2d(12, stride=12, ceil_mode=True),
        ))
#         resnet.avgpool = nn.AdaptiveAvgPool2d(2)
#         num_ftrs = net.fc.in_features
#         net.fc = nn.Linear(num_ftrs, 1)
#         self.net = net

    def forward(self, x):
#         x = self.net(self.bn(x))
        x = self.net(x)
        return x
    
model = Net([1, img_size, img_size])
    
# print(summary([3, 640, 640], model))

if use_gpu:
    model.cuda()

In [ ]:
lr = 0.0001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [ ]:
def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    exp_lr_scheduler.step()
    model.train()
    train_boneage_loss = 0
    train_gender_loss = 0
    train_loss = 0
    train_ccc = 0
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
#         print(output.size())
        
#         for i in xrange(output.size[0]):
#             if output[i,1] > 0.5:
#                 output[i,0] = (output[i,0] // 6) * 6
#             else:
#                 output[i,0] = (output[i,0] // 2) * 2

#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]
#         boneage_loss = F.l1_loss(boneage_out, target[:,0])
#         train_boneage_loss += boneage_loss.data[0]
#         gender_loss = F.binary_cross_entropy(F.sigmoid(output[:,1]), target[:,1])
#         train_gender_loss += gender_loss.data[0]
#         loss = 0.95 * boneage_loss + 0.05 * gender_loss
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
        loss = F.l1_loss(output.view(-1), target[:,0])
        train_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(output, target[:,0])
        train_ccc += c.data.cpu().numpy()[0]
        loss.backward()
        optimizer.step()
#         if batch_idx % (1000 // len(target)) == 0:
#     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBoneAge Loss: {:.4f}\tGender Loss: {:.4f}\t Loss: {:.4f}\tCCC: {:.4f}'.format(
#         epoch, batch_idx * len(data), len(train_loader),
#         100. * batch_idx / len(train_loader), train_boneage_loss / len(train_loader),
#         train_gender_loss / len(train_loader), train_loss / len(train_loader),
#         train_ccc / len(train_loader)))
    
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.4f}\tCCC: {:.4f}'.format(
        epoch, batch_idx * len(data), len(train_loader),
        100. * batch_idx / len(train_loader), train_loss / len(train_loader),
        train_ccc / len(train_loader)))

def test(test_loader):
    model.eval()
    test_boneage_loss = 0
    test_gender_loss = 0
    test_loss = 0
    test_ccc = 0
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
#         print(output.size())
#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]
#         print(boneage_out)
#         print(target[:,0])
#         boneage_loss = F.l1_loss(boneage_out, target[:,0])
#         print(boneage_loss)
#         test_boneage_loss += boneage_loss.data[0]
#         print(test_boneage_loss)
#         gender_loss = F.binary_cross_entropy(F.sigmoid(output[:,1]), target[:,1])
#         test_gender_loss += gender_loss.data[0]
#         loss = 0.95 * boneage_loss + 0.05 * gender_loss
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
        loss = F.l1_loss(output.view(-1), target[:,0])
        test_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(output, target[:,0])
        test_ccc += c.data.cpu().numpy()[0]

#     test_loss /= len(test_loader.dataset)
#     print('\nTest set: Avg BoneAge loss: {:.4f}, Avg Gender loss: {:.4f}, Avg loss: {:.4f} CCC: {:.4f}\n'.format(
#         test_boneage_loss / len(test_loader), test_gender_loss / len(test_loader),
#         test_loss / len(test_loader), test_ccc / len(test_loader)))
    
    print('\nTest set: Avg loss: {:.4f} CCC: {:.4f}\n'.format(
        test_loss / len(test_loader), test_ccc / len(test_loader)))

In [ ]:
# torch.save(model.state_dict(), '../models/9-27-1540-resnet50-1fold-5.7889-2.56.pth')

In [ ]:
for epoch in range(1, 100 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

In [ ]:
for epoch in range(1, 100 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

In [ ]:
for epoch in range(1, 100 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

In [ ]:
for epoch in range(1, 30 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])